In [26]:
from langchain.document_loaders import  UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from pymilvus import MilvusClient, DataType, Collection, connections
from langchain.chains import RetrievalQAWithSourcesChain
import langchain
from langchain.prompts import PromptTemplate
from langchain_milvus import Milvus # type: ignore
load_dotenv()

True

In [2]:
llm = ChatGroq(
    model="llama3-70b-8192",
    temperature=0.6
    )

In [3]:
urls=[
    "https://www.moneycontrol.com/news/business/tata-motors-mahindra-gain-certificates-for-production-linked-payouts-11281691.html",
    "https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html",
    "https://www.moneycontrol.com/news/business/stocks/buy-tata-motors-target-of-rs-743-kr-choksey-11080811.html"
]
loader = UnstructuredURLLoader(urls=urls)

In [4]:
data = loader.load()
len(data)

3

In [5]:
splitter = RecursiveCharacterTextSplitter(
    separators=['\n\n','\n',' '],
    chunk_size=300,
    chunk_overlap=30
)
chunks = splitter.split_documents(data)

In [20]:
messages = [
    (
        "system",
        f"{data[0].page_content}",
    ),
    ("human", "Read this passage and answer my question:\nWhat are the number of firms that will apply for certification?"),
]
ai_msg = llm.invoke(messages)
ai_msg.content

'According to the passage, the government expects 23 more firms to apply for certification by the end of September, in addition to the 6 companies that have already applied (including Tata Motors and Mahindra). Therefore, the total number of firms that will apply for certification is 29.'

In [6]:
encoder = HuggingFaceEmbeddings(model_name="all-MiniLM-L12-v2")

/home/arnab/Downloads/Equity-Research/genai-EqR/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [10]:
vectors = encoder.embed_documents([chunk.page_content for chunk in chunks],)
len(vectors),len(vectors[0])

(69, 384)

In [11]:
client = MilvusClient(
    uri="http://localhost:19530"
)


In [22]:
schema = MilvusClient.create_schema(
    auto_id=False,
    enable_dynamic_field=True,
)


In [31]:
{"source":"https://www.business-standard.com/companies/news/ats-homekraft-sells-340-flats-on-dwarka-expressway-gurugram-for-rs-825-cr-124101300264_1.html","text":"Access Denied\n\nYou don't have permission to access \"http://www.business-standard.com/companies/news/ats-homekraft-sells-340-flats-on-dwarka-expressway-gurugram-for-rs-825-cr-124101300264_1.html\" on this server.\n\nReference #18.d6cf5868.1728836210.45897004\n\nhttps://errors.edgesuite.net/18.d6cf5868.1728836210.45897004","pk":"BigInt_453134843223801872","vector":[-0.01095956,0.03666769,-0.062520355,0.055935208,0.036339186,0.008158711,-0.018924652,-0.061012056,-0.03158749,0.046843458,0.02689597,-0.065315366,0.006447088,0.00806536,0.01830609,0.06392956,0.072743714,-0.10658276,0.019678727,-0.008711872,0.031463306,-0.051888604,-0.035730507,-0.063714296,0.09013726,-0.0782858,-0.009141213,0.075466156,0.041468054,0.039571542,0.11865081,-0.046348587,-0.092184715,0.0006587549,0.036779206,0.004006465,0.039187633,-0.027124437,0.06514596,-0.017663755,0.054019623,0.028352153,0.025265068,0.05989461,0.0008404632,0.05899763,-0.009608672,-0.013629417,0.055848252,-0.076720014,-0.0005434441,0.010000166,0.017745918,-0.06650729,0.0022521815,0.017221125,-0.06672103,0.07850003,0.04838781,-0.054274116,0.082008794,-0.014116783,0.0028856432,-0.024678288,-0.037869934,-0.043895163,0.056181043,0.011493826,0.03171301,-0.073777996,-0.02982882,-0.008218034,-0.03210939,0.048228104,0.02663012,0.034386765,-0.057077754,0.05363721,0.028668333,-0.13943271,-0.042524632,0.048340026,0.09115387,0.082032554,-0.10654446,0.03840457,0.005833366,0.065328054,0.09973901,0.0595614,0.0923827,0.012445521,0.017047059,0.0019491018,0.020976933,-0.06671425,-0.044561848,-0.0128344335,-0.004234414,0.12871066,-0.0134246815,0.054411553,-0.13050383,-0.02013589,0.011658433,0.008636953,-0.030157745,0.11592352,-0.011918618,-0.024593629,0.03793838,-0.08704369,-0.021971684,0.054130994,-0.05646236,-0.014766727,-0.034646686,-0.014265521,-0.07553547,-0.018587152,-0.018473525,0.013247665,-0.04348581,-0.06496532,-0.0045287553,-0.08773447,0.060700323,0.07633917,-0.014302831,0.043479547,-0.01164571,-0.018272275,0.0071225083,-0.093359746,0.0054032584,-0.07570615,-0.0062375595,0.046435464,0.0122259045,0.013010028,-0.00610083,-0.09915335,0.046338078,0.014373985,0.09592611,-0.022116592,-0.048267305,0.08404382,0.059926186,0.029410454,0.0340493,0.048338033,-0.027293814,0.10598777,0.005735021,0.054527767,0.057474315,0.059306365,-0.0047152927,-0.019659193,0.004826869,0.020325474,0.06161949,0.03276556,0.030897066,-0.10334651,-0.05977967,-0.052039213,-0.015737316,-0.06965472,-0.08178711,-0.014507319,0.009417411,0.091316976,-0.051903784,0.010615584,0.07536175,0.011768919,-0.092615254,-0.028971802,-0.032638673,0.04341427,-0.054502517,0.024507366,-0.06034227,0.04991328,-0.047261253,0.050353825,0.016210468,-0.007747673,-0.04790996,-0.0323931,0.024417233,-0.13500187,0.031472452,-0.069282256,0.038473774,0.012657392,-0.020608969,-0.032924213,0.08476245,0.07800782,-0.05831234,0.007737499,-0.04789358,0.011139457,0.045299783,0.0151572805,-0.04186597,0.016049985,0.026848225,0.017146897,0.03273052,-0.008018203,-0.07075642,0.008180745,0.016428404,0.09174892,-0.0129279625,0.014451413,-0.058884595,0.059130747,-0.04961792,2.1105606e-32,-0.085356034,0.037617885,-0.11417552,-0.026984753,0.029296683,-0.011250329,-0.06464828,0.04795396,0.054068804,0.04916153,0.046281565,0.07937381,0.031082535,-0.070928894,0.008169115,-0.0155500015,0.026954185,-0.039152868,-0.087704904,0.05209922,-0.08115882,0.009817617,-0.041562807,0.0132688545,0.02538858,0.11930274,0.038240746,0.008486431,-0.068776116,0.014897093,0.06339566,-0.06254685,-0.07271924,0.057844862,-0.009201633,-0.008478749,0.08735302,0.029770598,-0.018100074,-0.0044608833,0.028955746,0.012768151,0.06497171,-0.050154284,-0.07754195,-0.07686859,0.019233016,-0.050106294,-0.015863784,0.049784426,0.04487038,0.018921666,0.09859019,-0.009242119,0.03229394,0.079176,0.035860993,-0.01473553,0.08067145,-0.004478083,0.01372848,-0.01953654,-0.08986976,0.06750002,-0.03242843,-0.029961016,0.031140517,-0.024804726,0.0389302,-0.052203927,-0.08204884,-0.046312574,0.008817507,-0.0722157,0.009821191,-0.047450613,0.044738524,0.04008375,0.032739345,-0.022635026,0.014830563,0.00094427477,0.03197195,-0.024463786,0.058483426,-0.0037539306,0.095870554,0.036714736,-0.0045226635,-0.01656288,-0.014169345,0.016073672,-0.08043405,0.14426266,-0.0009312453,5.439049e-32,-0.04830755,-0.08927223,-0.05820544,-0.004212035,-0.004144963,-0.09641774,0.012765675,0.012919734,-0.017692655,-0.0057494016,-0.012714328,-0.067592084,-0.10613013,0.016952945,0.010795733,-0.0391709,0.0049941395,-0.041982636,-0.009864363,-0.016544892,0.044183902,-0.032845326,0.09319307,0.015086456,-0.08049502,-0.03954226,-0.018186953,-0.023567155,0.06546053,-0.033425547,-0.040083107,0.042754043,-0.025991721,-0.06134158,-0.0005425217,0.004353175,0.04526531,-0.011504959,-0.058791526,0.090880424,-0.034669828,0.0012406817,0.08394466,0.0042951973,0.0016475355,0.011252309,0.023648534,0.06387729,0.019829597,-0.012084296,-0.020852365,-0.07989419,-0.012316114,-0.03843911,-0.007296388,0.019809108,-0.025079846,-0.06929233,-0.0603627,0.034629833,0.023353131,0.021468587,-0.01581168,0.0795329]}['text']

'Access Denied\n\nYou don\'t have permission to access "http://www.business-standard.com/companies/news/ats-homekraft-sells-340-flats-on-dwarka-expressway-gurugram-for-rs-825-cr-124101300264_1.html" on this server.\n\nReference #18.d6cf5868.1728836210.45897004\n\nhttps://errors.edgesuite.net/18.d6cf5868.1728836210.45897004'

In [23]:
schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="source_id",datatype=DataType.INT16)
schema.add_field(field_name="vector", datatype=DataType.FLOAT_VECTOR, dim=len(vectors[0]))


{'auto_id': False, 'description': '', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'source_id', 'description': '', 'type': <DataType.INT16: 3>}, {'name': 'vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 384}}], 'enable_dynamic_field': True}

In [24]:
client.create_collection(
    collection_name="finance_articles", 
    schema=schema, 
)

In [25]:
index_params = MilvusClient.prepare_index_params()
index_params.add_index(
    field_name="vector",
    metric_type="COSINE",
    index_type="FLAT",
    index_name="article_vectors"
)


In [26]:
client.create_index(
    collection_name="finance_articles",
    index_params=index_params,
    sync=False # Whether to wait for index creation to complete before returning. Defaults to True.
)



In [28]:
res = client.describe_index(
    collection_name="finance_articles",
    index_name="article_vectors"
)
print(res)


{'index_type': 'FLAT', 'metric_type': 'COSINE', 'field_name': 'vector', 'index_name': 'article_vectors', 'total_rows': 0, 'indexed_rows': 0, 'pending_index_rows': 0, 'state': 'Finished'}


In [29]:
source_url_map = {i:urls[i] for i in range(len(urls))}
url_source_map = {v:k for k,v in source_url_map.items()}

In [30]:
vector_data = []
for i in range(len(vectors)):
    vector_data.append({'id':i,
                        'source_id':url_source_map[chunks[i].metadata['source']],
                        'vector':vectors[i]})

In [31]:
res = client.insert(
    collection_name="finance_articles",
    data=vector_data
)



In [11]:
question = "What are the number of firms that will apply for certification by the end of September ?"
query = encoder.embed_query(question)

In [35]:
client.load_collection("finance_articles")

In [37]:
res = client.search(
    collection_name="finance_articles", # Replace with the actual name of your collection
    # Replace with your query vector
    data=[query],
    limit=5, # Max. number of search results to return
    search_params={"metric_type": "COSINE", "params": {}}, # Search parameters
    output_fields=['source_id']
)


In [41]:
res[0]

[{'id': 9, 'distance': 0.7440287470817566, 'entity': {'source_id': 0}},
 {'id': 4, 'distance': 0.43733227252960205, 'entity': {'source_id': 0}},
 {'id': 7, 'distance': 0.42282533645629883, 'entity': {'source_id': 0}},
 {'id': 5, 'distance': 0.39404892921447754, 'entity': {'source_id': 0}},
 {'id': 8, 'distance': 0.3491817116737366, 'entity': {'source_id': 0}}]

In [15]:
connections.connect(
  alias="default", 
  host='localhost', 
  port='19530'
)

In [16]:
total_records = Collection("finance_articles").num_entities
total_records

69

In [7]:
index_params = {
    "index_type": 'IVF_FLAT',
    "params": {"nlist": 128},
    "metric_type": "COSINE"
}

In [29]:
vector_store = Milvus(
    embedding_function=encoder,
    collection_name="finance_articles",
    auto_id=True,
    index_params=index_params,
    connection_args={"uri":"https://in03-a4e6155e8e44f89.serverless.gcp-us-west1.cloud.zilliz.com",
                     "token":"0277afb5c62783882bbf2b6e32fe17a2c5111fb956fec6b338af16a58d84124a94095656ecb23192f3647f0d15a0ebc269ce904a",
                     "secure": True}
)

In [30]:
vector_store.add_documents(chunks)

[453134843223800233,
 453134843223800234,
 453134843223800235,
 453134843223800236,
 453134843223800237,
 453134843223800238,
 453134843223800239,
 453134843223800240,
 453134843223800241,
 453134843223800242,
 453134843223800243,
 453134843223800244,
 453134843223800245,
 453134843223800246,
 453134843223800247,
 453134843223800248,
 453134843223800249,
 453134843223800250,
 453134843223800251,
 453134843223800252,
 453134843223800253,
 453134843223800254,
 453134843223800255,
 453134843223800256,
 453134843223800257,
 453134843223800258,
 453134843223800259,
 453134843223800260,
 453134843223800261,
 453134843223800262,
 453134843223800263,
 453134843223800264,
 453134843223800265,
 453134843223800266,
 453134843223800267,
 453134843223800268,
 453134843223800269,
 453134843223800270,
 453134843223800271,
 453134843223800272,
 453134843223800273,
 453134843223800274,
 453134843223800275,
 453134843223800276,
 453134843223800277,
 453134843223800278,
 453134843223800279,
 453134843223

In [22]:
question = "What are the total number of firms that will apply for certification by the end September?"
vector_store.similarity_search(question,k=4,expr=f"source in {urls[1:2]}")

[Document(metadata={'source': 'https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html', 'pk': 453134843223799707}, page_content='Forum\n\nMC 30\n\nNews\n\nBusiness\n\nMarkets\n\nStocks\n\nIncome Tax Calculator\n\nElection Schedule 2024\n\nIndia News\n\nEconomy\n\nMutual Funds\n\nPersonal Finance\n\nIPO News\n\nStartups\n\nStocks: A | B | C | D | E | F | G | H | I | J | K | L | M | N | O | P | Q | R | S | T | U | V | W | X | Y | Z | Others'),
 Document(metadata={'source': 'https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html', 'pk': 453134843223799711}, page_content='Overdrive\n\nTopper Learning\n\nAbout us | Contact Us | Advisory Alert | Advertise with Us | Support | Disclaimer | Privacy Policy | Cookie Policy | Terms & Conditions | Careers | Financial Terms (Glossary) | FAQs | Sitemap | RSS Feed | Investors'),
 Document(metadata={'source': 'https://www.moneycon

In [23]:
PROMPT_TEMPLATE = """Human: You are an AI assistant, and provides answers to questions by using fact based and statistical information when possible.
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags and avoid using first person pronouns.
If you don't know the answer, just say that you don't know, don't try to make up an answer or just say you cannot find it if it is really not present there.
<context>
{summaries}
</context>

<question>
{question}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:
"""
prompt = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["summaries", "question"]
)

In [24]:
chain = RetrievalQAWithSourcesChain.from_chain_type(llm=llm,
                                             retriever=vector_store.as_retriever(search_kwargs={'k':2,'expr':f"source in {urls[:2]}"}),
                                             chain_type="stuff",
                                             return_source_documents=True,
                                             chain_type_kwargs={"prompt":prompt}
                                             )

In [25]:
question = "What are the total number of firms that will apply for certification by the end of January 2025 ?"
chain(question)

{'question': 'What are the total number of firms that will apply for certification by the end of January 2025 ?',
 'answer': 'According to the provided context, the government expects 23 more firms to apply for the certification by the end of September 2023, in addition to the 6 firms that have already applied (including Tata Motors and Mahindra). There is no information provided about the total number of firms that will apply for certification by the end of January 2025.',
 'sources': '',
 'source_documents': [Document(metadata={'source': 'https://www.moneycontrol.com/news/business/tata-motors-mahindra-gain-certificates-for-production-linked-payouts-11281691.html', 'pk': 453134843223799679}, page_content='Apart from Tata Motors and Mahindra, four other companies have applied for certification which is crucial for receiving the incentives, the government said, without naming the companies.\n\nThe government also said it expects 23 more firms to apply for the certification by the end of

In [31]:
langchain.debug=False